In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import k-independent data
synapses = pd.read_csv('data/synapses_w_ids.csv', index_col=0)
cells = pd.read_csv('data/cells_no_repeats.csv', index_col=0)
print('Synapses: ', synapses.shape)
print('Cells: ', cells.shape)
display(synapses.head())
display(cells.head())

Synapses:  (5421809, 18)
Cells:  (56209, 5)


,pre_pt_root_id,post_pt_root_id,size,cell_type_pre,cb_x_pre,cb_y_pre,cb_z_pre,cell_type_post,cb_x_post,cb_y_post,cb_z_post,cb_x_diff,cb_y_diff,cb_z_diff,ctr_pt_x,ctr_pt_y,ctr_pt_z,synapse_id
0,864691135564683351,864691136811959667,960,23P,557.248,570.560,732.52,23P,791.744,483.968,780.44,-234.496,86.592,-47.92,637.608,371.352,720.20,0
1,864691135614119115,864691135508912649,7576,23P,774.720,504.960,895.68,23P,807.936,459.584,870.28,-33.216,45.376,25.40,805.376,488.376,863.60,1
3,864691135113147801,864691136022555784,684,23P,883.072,451.456,817.84,23P,851.520,483.840,759.96,31.552,-32.384,57.88,858.328,516.648,775.88,3
4,864691135194393642,864691135341362885,23172,23P,781.248,449.984,696.88,23P,798.720,465.152,758.56,-17.472,-15.168,-61.68,789.400,478.040,691.00,4
5,864691136272938174,864691135683554546,3660,23P,762.368,473.792,773.68,23P,820.352,446.784,719.08,-57.984,27.008,54.60,756.624,440.928,710.60,5


,cell_type,pt_root_id,pt_x,pt_y,pt_z
Unnamed: 0,,,,,
0,23P,864691135639004475,71136,110528,20220
1,23P,864691135771677771,72576,108656,20291
2,23P,864691135864089470,79632,121456,16754
3,23P,864691135560505569,80128,124000,16563
4,23P,864691136315868311,80144,126928,16622


In [3]:
synapses.set_index('synapse_id', inplace=True)
display(synapses.head())

,pre_pt_root_id,post_pt_root_id,size,cell_type_pre,cb_x_pre,cb_y_pre,cb_z_pre,cell_type_post,cb_x_post,cb_y_post,cb_z_post,cb_x_diff,cb_y_diff,cb_z_diff,ctr_pt_x,ctr_pt_y,ctr_pt_z
synapse_id,,,,,,,,,,,,,,,,,
0,864691135564683351,864691136811959667,960,23P,557.248,570.560,732.52,23P,791.744,483.968,780.44,-234.496,86.592,-47.92,637.608,371.352,720.20
1,864691135614119115,864691135508912649,7576,23P,774.720,504.960,895.68,23P,807.936,459.584,870.28,-33.216,45.376,25.40,805.376,488.376,863.60
3,864691135113147801,864691136022555784,684,23P,883.072,451.456,817.84,23P,851.520,483.840,759.96,31.552,-32.384,57.88,858.328,516.648,775.88
4,864691135194393642,864691135341362885,23172,23P,781.248,449.984,696.88,23P,798.720,465.152,758.56,-17.472,-15.168,-61.68,789.400,478.040,691.00
5,864691136272938174,864691135683554546,3660,23P,762.368,473.792,773.68,23P,820.352,446.784,719.08,-57.984,27.008,54.60,756.624,440.928,710.60


In [44]:
# Specify k
k = 6

In [45]:
# Import k-dependent data
clusters = pd.read_csv(f'data/synapse_clusters/clusters_ind_cntr_k{k}.csv', index_col=0)
clusters.set_index('cluster_id', inplace=True)
print('Clusters: ', clusters.shape)
display(clusters.head())

Clusters:  (64330, 12)


,synapse_id_0,synapse_id_1,synapse_id_2,synapse_id_3,synapse_id_4,synapse_id_5,density_score,post_pt_root_id,post_cell_type,centroid_x,centroid_y,centroid_z
cluster_id,,,,,,,,,,,,
0,941001,900422,906864,932560,932857,923889,0.397512,864691134884741370,4P,572.974667,691.796667,1052.113333
1,958004,979837,932857,932560,912217,906864,0.413748,864691134884741370,4P,577.322667,692.268000,1054.240000
2,674327,704519,662648,676483,709257,678862,0.384314,864691134884742906,6P-IT,488.466000,875.476000,936.273333
3,687488,689522,657710,670465,673155,692136,0.379345,864691134884742906,6P-IT,553.638667,1000.280667,1026.933333
4,685121,2241694,711425,686754,702105,676480,0.381143,864691134884743930,6P-IT,586.533333,758.324000,750.860000


In [46]:
unique_synapses = set()
for ij in range(k):
    unique_synapses.update(clusters[f'synapse_id_{ij}'])
print(len(unique_synapses))
print(clusters.shape[0] * k)

316685
385980


In [47]:
cluster_synapses = synapses.loc[list(unique_synapses)]
pre_cb_positions = cluster_synapses.loc[:, ['cb_x_pre', 'cb_y_pre', 'cb_z_pre']]
print(cluster_synapses.shape)
print(pre_cb_positions.shape)
display(pre_cb_positions.head())

(316685, 17)
(316685, 3)


,cb_x_pre,cb_y_pre,cb_z_pre
synapse_id,,,
3145728,779.072,754.944,1052.52
6291457,1240.384,842.240,800.32
4194308,1088.448,716.864,776.64
4194309,1037.440,752.128,946.76
4194315,1059.712,790.912,895.60


In [48]:
id_keys = [f'synapse_id_{m}' for m in range(k)]
synapse_ids = clusters[id_keys]
display(synapse_ids.head())

,synapse_id_0,synapse_id_1,synapse_id_2,synapse_id_3,synapse_id_4,synapse_id_5
cluster_id,,,,,,
0,941001,900422,906864,932560,932857,923889
1,958004,979837,932857,932560,912217,906864
2,674327,704519,662648,676483,709257,678862
3,687488,689522,657710,670465,673155,692136
4,685121,2241694,711425,686754,702105,676480


In [49]:
# Get the cell body positions for each synapse in each cluster
col_names = [[f'pre_cb_x_{m}', f'pre_cb_y_{m}', f'pre_cb_z_{m}'] for m in range(k)]
for cluster_id in synapse_ids.index:
    cur_synapse = synapse_ids.loc[cluster_id, :]
    for n in range(k):
        cur_id = cur_synapse[f'synapse_id_{n}']
        cur_cb_pos = pre_cb_positions.loc[cur_id, :]
        col_name = col_names[n]
        clusters.loc[cluster_id, col_name] = cur_cb_pos.values

print(clusters.shape)
display(clusters.head())
    

(64330, 30)


,synapse_id_0,synapse_id_1,synapse_id_2,synapse_id_3,synapse_id_4,synapse_id_5,density_score,post_pt_root_id,post_cell_type,centroid_x,...,pre_cb_z_2,pre_cb_x_3,pre_cb_y_3,pre_cb_z_3,pre_cb_x_4,pre_cb_y_4,pre_cb_z_4,pre_cb_x_5,pre_cb_y_5,pre_cb_z_5
cluster_id,,,,,,,,,,,,,,,,,,,,,
0,941001,900422,906864,932560,932857,923889,0.397512,864691134884741370,4P,572.974667,...,996.24,571.648,688.448,1058.40,489.536,612.352,1085.92,686.464,969.728,957.56
1,958004,979837,932857,932560,912217,906864,0.413748,864691134884741370,4P,577.322667,...,1085.92,571.648,688.448,1058.40,656.192,696.064,1035.20,593.600,592.704,996.24
2,674327,704519,662648,676483,709257,678862,0.384314,864691134884742906,6P-IT,488.466000,...,949.80,500.352,858.240,1026.04,485.952,853.376,882.56,570.112,1040.448,996.24
3,687488,689522,657710,670465,673155,692136,0.379345,864691134884742906,6P-IT,553.638667,...,923.20,570.432,1053.568,1045.96,500.224,1012.352,998.88,503.104,1008.512,1037.72
4,685121,2241694,711425,686754,702105,676480,0.381143,864691134884743930,6P-IT,586.533333,...,713.84,608.704,620.160,728.32,553.600,654.336,758.08,658.880,713.600,696.48


In [50]:
clusters.to_csv(f'data/synapse_clusters/clusters_w_pre_pos_k{k}.csv')